# Differentiation - Homework

__Métodos Intensivos de Computación Estadística__

Juan Sebastián Corredor Rodriguez - jucorredorr@unal.edu.co

See my [Github Account](https://github.com/juanse1608) to know more about me and my projects.

## Introduction

Los datos del archivo de Binary Matrix abstracts.csv contiene la matrix document-término del conjunto de datos de abstracts de la tarea 1. Asuma la dimensión $m$ y haga un anáalisis de compontes principales y estime los vectores $\theta_i$ donde $i = 1, \dots N$ dados por las primeras $m$ componentes principales. Use las herramientas de diferenciación automática y numérica disponibles en su lenguaje de programación de preferencia. Incluya todos sus códigos por separado.

Let's do the exercise with $m=5$.

In [2]:
#Loading the required libraries
import pandas as pd
import numpy as npx
import matplotlib.pyplot as plt
import autograd as grad

from sklearn.preprocessing import OneHotEncoder
import autograd.numpy as np
from autograd import elementwise_grad as egrad 
from sklearn.decomposition import PCA

In [3]:
%cd ../../Datasets/

/Users/JuanSebastianCorredorRodriguez/Documents/Git Repositories/Jupyters-2019/Jupyters/Datasets


In [43]:
dta = pd.read_csv('Binary_Matrix_abstracts.csv', index_col = 0)
m = 5
pca = PCA(n_components=m)
pca.fit(dta)

#Each row is one of the theta_i
y = pca.fit_transform(dta)

In [9]:
#Let's see y
print(y.shape)

(29518, 5)


Note that
$$\triangledown \beta = \frac{\partial L}{\partial \beta} = \sum_{i=1}^{5}\frac{\partial l_{ij}}{\partial \beta_j} = \sum_{i=1}^{5} (y_{ij} - \pi_{ij}) \begin{bmatrix} \theta_i \\ 1 \end{bmatrix}$$

The probabilities $\pi_{ij}$ can be obtained with the cdf of the logistic distribution.

In [25]:
#Logistic Distribution
def logistic_distribution(x):
    return 1/(1+np.exp(-x))

## Numeric and Auto Differentiation

Let's implements the Numeric and Auto-Diff with ``autograd`` library.

-----

In [11]:
def log_ij(y_ij, pi_ij):
    return y_ij* np.log(pi_ij) + (1-y_ij) * np.log(1-pi_ij)

def probability_ij(eta):
    return logistic_distribution(eta)

def eta_function(a_j, theta_i, d_j):
    return (np.dot(a_j.T, theta_i) - d_j)

In [23]:
#Auto Differentiation
a = grad.grad(log_ij, 1)
b = grad.grad(probability_ij, 0)

x = np.zeros(y.shape)

#Se calcula la derivada para cada i,j de la matriz Y
for i in range(len(y)):
    theta_i = y[i,:]
    a_j = np.ones((y.shape[1],1))
    d_j = 0
    for j in range(y.shape[1]):
        eta_j = np.dot(a_j.T, theta_i) - d_j
        x[i,j] = b(eta_j) * a(y[i,j], probability_ij(eta_j))

In [31]:
#Numeric Differentiation
h = 0.0000001
x_2 = np.zeros(y.shape)
#Se calcula la derivada numérica

for i in range(len(y)):
    theta_i = y[i,:].T
    a_j = np.ones((y.shape[1],1))
    d_j = 0
    for j in range(y.shape[1]):
        eta_j = np.dot(a_j.T, theta_i) - d_j
        b = 1/h * (probability_ij(eta_j+h) - probability_ij(eta_j))
        a = 1/h * (log_ij(y[i,j], probability_ij(eta_j)+h)  -  log_ij(y[i,j], probability_ij(eta_j)))
        x_2[i,j] = b*a

In [33]:
#The true derivate
x_3 = np.zeros(y.shape)
#Se calcula la derivada numérica

for i in range(len(y)):
    theta_i = y[i,:].T
    a_j = np.ones((y.shape[1],1))
    d_j = 0
    for j in range(y.shape[1]):
        eta_j = np.dot(a_j.T, theta_i) - d_j
        p = probability_ij(eta_j)
        x_3[i,j] = (y[i,j] - p)/(p*(1-p)) * np.exp(-eta_j)/np.power(np.exp(-eta_j)+1, 2)

Numeric Differentiation take less time that Auto Differentiation but now let's look at the errors.

In [38]:
#Let's see the MSE 
diff_auto = np.sum(np.power(x-x_3,2))
diff_num = np.sum(np.power(x_2-x_3,2))
print('MSE Automatic Diff: {}'.format(diff_auto))
print('MSE Numeric Diff: {}'.format(diff_num))
if diff_auto < diff_num:
    print('\nAutomatic Differentiation MSE is smaller than Numeric Differentiation')
else:
    print('\nNumeric Differentiation MSE is smaller than Automatic Differentiation')

MSE Automatic Diff: 1.637656801626484e-27
MSE Numeric Diff: 3.818927030738308e-09

Automatic Differentiation MSE is smaller than Numeric Differentiation


## Information Matrix

Let's calculate the information matrix

---

In [44]:
d_eta_a = grad.grad(eta_function, 0)
d_eta_d = grad.grad(eta_function, 2)
J_1 = np.zeros((m+1,m+1,y.shape[0],y.shape[1]))
J_2 = np.zeros((m+1,m+1,y.shape[0],y.shape[1]))
J_3 = np.zeros((m+1,m+1,y.shape[0],y.shape[1]))

for i in range(len(y)):
    theta_i = y[i,:].T.reshape((y.shape[1], 1))
    a_j = np.ones((y.shape[1], 1))
    d_j = 0
    T = np.append(theta_i,1).reshape((y.shape[1] + 1, 1))
    for j in range(y.shape[1]):
        J_1[:,:, i,j] = x[i,j]*np.dot(T,T.T)
        J_2[:,:, i,j] = x_2[i,j]*np.dot(T,T.T)
        J_3[:,:, i,j] = x_3[i,j]*np.dot(T,T.T)

In [47]:
J_1_b1 = 0
J_2_b1 = 0
J_3_b1 = 0

for i in range(len(y)):
    J_1_b1 = J_1_b1 + J_1[:,:,i,1]
    J_2_b1 = J_2_b1 + J_2[:,:,i,1]    
    J_3_b1 = J_3_b1 + J_3[:,:,i,1]   

In [50]:
#Let's the MSE of Numeric adn Automatic
print(np.sum(np.power(J_1_b1 - J_3_b1,2)))
print(np.sum(np.power(J_2_b1 - J_3_b1,2)))
print(np.sum(np.power(J_1_b1 - J_3_b1,2)))

9.697157874407051e-24
8.93355401686857e-06
9.697157874407051e-24


It is clear the superiority of Automatic Differentiation in terms of error against its rivals.